In [176]:
from bs4 import BeautifulSoup
import requests
import math
from dateutil import parser
import html
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains


In [116]:
def scrapCorpus(sources,keyword,nb_docs):

    source = ""
    position = ""
    company = ""
    workplace = ""
    published_date = ""
    contract_type = ""
    long_infos = ""
    # 20 offres par pages
    pages = math.ceil(nb_docs/20)

    if  "emploi-territorial" in sources:
        source = "emploi-territorial"
        rootLink = "https://www.emploi-territorial.fr"
        # 20 offres par pages
        # pages = math.ceil(nb_docs/20)
        url = f"{rootLink}/emploi-mobilite/?adv-search={keyword}&page={pages}"
        response = requests.get(url) 
        soup = BeautifulSoup(response.content, 'html.parser')
        root = soup.find("body")
        offresLinkElems = root.select("div[class*='bloc-lien-offre'] > a[class*='lien-details-offre']")[:nb_docs]    
        links = [rootLink+offresLinkElem.get("href") for offresLinkElem in offresLinkElems]

        corpus = list(dict())

        for link in links:
            response = requests.get(link)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            root = soup.find("body")

            position = root.select("h2[class*='set-line-emploi']")[0].text
            company = root.select("div[class*='offre-item-value'] > strong > a")[0].text if root.select("div[class*='offre-item-value'] > strong > a") else "NULL"
            workplace = root.select_one("div[class*='offre-item-label']:contains('Lieu de travail') + .offre-item-value").text if root.select_one("div[class*='offre-item-label']:contains('Lieu de travail') + .offre-item-value") else "NULL"
            published_date = root.select_one("div[class*='px-3']:contains('Publiée le') > .set-color-green").text if root.select_one("div[class*='px-3']:contains('Publiée le') > .set-color-green") else "NULL"
            contract_type = root.select_one('div[class*="offre-item-label"]:contains("Type d\'emploi") + .offre-item-value').text if root.select_one('div[class*="offre-item-label"]:contains("Type d\'emploi") + .offre-item-value') else "NULL"
            
            long_text = root.select('div[class*="offre-item-text"]')

            long_infos = " ".join([long_text[0].text,long_text[1].text,long_text[2].text])

            corpus.append({"source":source,"position":position,"company":company,"workplace":workplace,"published_date":published_date,"contract_type":contract_type,"description":long_infos})
    
    if "apec" in sources:
        source = "apec"
        service = ChromeService("C:/Users/leogo/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe")
        driver = webdriver.Chrome(service=service)
        try:
            for page in range(pages):    
                try:
                    driver.get(f'https://www.apec.fr/candidat/recherche-emploi.html/emploi?motsCles={keyword}&page={pages}')
                    # Initial find of elements
                    try:
                        deny_cookies_button = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#onetrust-reject-all-handler'))
                        )

                        deny_cookies_button.click()
                    except TimeoutException:
                        pass

                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-offer"))
                    )

                    # Getting docs left if last page to query
                    if nb_docs%20 != 0 and page==pages-1: 
                        limit = nb_docs%20+1
                        div_elements_to_click_list = driver.find_elements(By.CSS_SELECTOR, 'div.card-offer')[:limit]
                    else :
                        div_elements_to_click_list = driver.find_elements(By.CSS_SELECTOR, 'div.card-offer')
                        
                    for index in range(len(div_elements_to_click_list)):
                        try:
                            WebDriverWait(driver, 10).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-offer"))
                            )
                            # Re-find elements after navigating back
                            div_elements_to_click_list = driver.find_elements(By.CSS_SELECTOR, 'div.card-offer')

                            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                            WebDriverWait(driver, 10).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-offer"))
                            )
                            # Check if the index is within the valid range

                            div_elements_to_click = div_elements_to_click_list[index]
                            company = div_elements_to_click.find_element(By.CSS_SELECTOR, 'p.card-offer__company').text
                            contract_type = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[0].text
                            workplace = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[1].text
                            published_date = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[2].text
                            # Scroll into view

                            actions = ActionChains(driver)
                            actions.move_to_element(div_elements_to_click).click().perform()

                            # Wait for the child element to become present in the DOM
                            WebDriverWait(driver, 20).until(
                                EC.presence_of_element_located((By.XPATH, f"//h4[text()='Descriptif du poste']"))
                            )
                            # Get the page source after the click
                            page_source = driver.page_source

                            # Use Beautiful Soup to parse the page source
                            soup = BeautifulSoup(page_source, 'html.parser')

                            # Example: Retrieve the text of a specific element
                            desc = soup.select("h4:contains('Descriptif du poste') + p")
                            descText = ["".join(elem.text) for elem in desc][0]

                            profile = soup.select("h4:contains('Profil recherché') + p")
                            profileText = ["".join(elem.text) for elem in profile][0]                  

                            position = soup.select_one("h4:contains('Métier') + span").text

                            long_infos = " ".join([descText,profileText])
                            
                            corpus.append({"source":source,"position":position,"company":company,"workplace":workplace,"published_date":published_date,"contract_type":contract_type,"description":long_infos})
                        except Exception as e:
                            # Print the exception for debugging purposes
                            print(f"Error: {e}")

                        finally:
                            # Navigate back to the main page
                            driver.execute_script("window.history.go(-1);")

                except Exception as e:
                    # Print the exception for debugging purposes
                    print(f"Error: {e}")

        finally:
            driver.quit()
    return(corpus)


In [117]:
corpus = scrapCorpus(sources=["emploi-territorial","apec"],keyword="data",nb_docs=22)

Error: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79DFE82B2+55298]
	(No symbol) [0x00007FF79DF55E02]
	(No symbol) [0x00007FF79DE105AB]
	(No symbol) [0x00007FF79DE5175C]
	(No symbol) [0x00007FF79DE518DC]
	(No symbol) [0x00007FF79DE8CBC7]
	(No symbol) [0x00007FF79DE720EF]
	(No symbol) [0x00007FF79DE8AAA4]
	(No symbol) [0x00007FF79DE71E83]
	(No symbol) [0x00007FF79DE4670A]
	(No symbol) [0x00007FF79DE47964]
	GetHandleVerifier [0x00007FF79E360AAB+3694587]
	GetHandleVerifier [0x00007FF79E3B728E+4048862]
	GetHandleVerifier [0x00007FF79E3AF173+4015811]
	GetHandleVerifier [0x00007FF79E0847D6+695590]
	(No symbol) [0x00007FF79DF60CE8]
	(No symbol) [0x00007FF79DF5CF34]
	(No symbol) [0x00007FF79DF5D062]
	(No symbol) [0x00007FF79DF4D3A3]
	BaseThreadInitThunk [0x00007FFF1E43257D+29]
	RtlUserThreadStart [0x00007FFF1E82AA58+40]



In [196]:
import mysql.connector

# Replace these values with your database connection details
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'job_scrapping'
}

def call_dynamic_insert_procedure(cursor, procedure_name, param_dict):
    try:
        # Prepare the SQL statement with placeholders for parameters
        placeholders = ', '.join(['%s'] * len(param_dict))
        sql = f"CALL {procedure_name}({placeholders})"
        print(sql)

        # Execute the SQL statement with the values from the dictionary
        cursor.callproc(procedure_name, list(param_dict.values()))

        # Commit the changes
        conn.commit()

    except mysql.connector.Error as err:
        print(f"Error: {err}")
# Connect to the database
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Call the generic stored procedure

def insertTable(table_name, column_name, items):
    for item in items:
        parameters = {'pTableName': table_name, 'pColumnNames': column_name, 'pColumnValues': f'"{item}"'}
        call_dynamic_insert_procedure(cursor, "pInsertDW", parameters)   

def refreshDW():
    try:
        cursor.callproc('pTruncateDW')
        insertTable("d_website", 'label', set([item["source"].upper() for item in corpus]))
        insertTable("d_contract_type", 'contract_type', set([item["contract_type"].upper() for item in corpus]))
        insertTable("d_company", 'label', set([item["company"].upper() for item in corpus]))
        insertTable("d_city", 'city', set([item["workplace"].upper() for item in corpus]))
    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()


refreshDW()

CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
CALL pInsertDW(%s, %s, %s)
C

In [150]:
list(set([item["company"].upper() for item in corpus]))

["OPT'IN RECRUTEMENT",
 'OEB',
 'STUDIO RH',
 'ADEQUATION',
 'FITEC',
 'CONSEIL DEPARTEMENTAL DE MEURTHE ET MOSELLE',
 'KAISENS DATA',
 'ASSYSTEM',
 'NULL',
 'HELLOWORK',
 'CONSEIL DEPARTEMENTAL DES PYRÉNÉES-ATLANTIQUES',
 'GRAVOTECH MARKING',
 'CONSEIL REGIONAL OCCITANIE ',
 "GRAND MONTAUBAN COMMUNAUTÉ D'AGGLOMÉRATION",
 'CONSEIL DEPARTEMENTAL DE  LA SEINE MARITIME',
 'METROPOLE AIX-MARSEILLE-PROVENCE',
 'CHAVILLE',
 'CONSEIL DÉPARTEMENTAL DU CALVADOS',
 'DÉPARTEMENT DES HAUTS-DE-SEINE',
 'REGION GRAND EST',
 'ORGANISATION PROFESSIONNELLE DE LA PREVENTION DANS LE BTP',
 'METEOJOB',
 'GESER BEST',
 "COLLECTIVITE EUROPEENNE D'ALSACE",
 'MICHAEL PAGE INTERNATIONAL FRANCE',
 'METROPOLE LYON',
 'BUSINESS & DECISION CORPORATE SERVICES',
 'NOISY-LE-GRAND',
 'GRENOBLE-ALPES MÉTROPOLE']

In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains


service = ChromeService("C:/Users/leogo/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe")
key_word = "data"
driver = webdriver.Chrome(service=service)
corpustest = list()
try:
    for page in range(pages):    
        try:
            driver.get(f'https://www.apec.fr/candidat/recherche-emploi.html/emploi?motsCles={key_word}&page={page}')
            # Initial find of elements
            try:
                deny_cookies_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#onetrust-reject-all-handler'))
                )

                deny_cookies_button.click()
            except TimeoutException:
                pass

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-offer"))
            )
            div_elements_to_click_list = driver.find_elements(By.CSS_SELECTOR, 'div.card-offer')
            for index in range(len(div_elements_to_click_list)):
                try:
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-offer"))
                    )
                    # Re-find elements after navigating back
                    div_elements_to_click_list = driver.find_elements(By.CSS_SELECTOR, 'div.card-offer')

                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-offer"))
                    )
                    # Check if the index is within the valid range

                    div_elements_to_click = div_elements_to_click_list[index]
                    company = div_elements_to_click.find_element(By.CSS_SELECTOR, 'p.card-offer__company').text
                    contract_type = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[0].text
                    workplace = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[1].text
                    published_date = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[2].text
                    # Scroll into view

                    actions = ActionChains(driver)
                    actions.move_to_element(div_elements_to_click).click().perform()

                    # Wait for the child element to become present in the DOM
                    WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located((By.XPATH, f"//h4[text()='Descriptif du poste']"))
                    )
                    # Get the page source after the click
                    page_source = driver.page_source

                    # Use Beautiful Soup to parse the page source
                    soup = BeautifulSoup(page_source, 'html.parser')

                    # Example: Retrieve the text of a specific element
                    desc = soup.select("h4:contains('Descriptif du poste') + p")
                    descText = ["".join(elem.text) for elem in desc][0]

                    profile = soup.select("h4:contains('Profil recherché') + p")
                    profileText = ["".join(elem.text) for elem in profile][0]                  

                    position = soup.select_one("h4:contains('Métier') + span").text

                    corpustest.append({"position":position,"company":company,"workplace":workplace,"published_date":published_date,"contract_type":contract_type,"description":" ".join([descText,profileText])})

                except Exception as e:
                    # Print the exception for debugging purposes
                    print(f"Error: {e}")

                finally:
                    # Navigate back to the main page
                    driver.execute_script("window.history.go(-1);")

        except Exception as e:
            # Print the exception for debugging purposes
            print(f"Error: {e}")

finally:
    driver.quit()

page = 0
page = 1


In [84]:
corpustest

[{'position': 'Data engineer',
  'company': '(CONFIDENTIELLE)',
  'workplace': 'Seine-Saint-Denis - 93',
  'published_date': '21/11/2023',
  'contract_type': 'CDI',
  'description': 'Description du posteNous recrutons un Data Analyst / Data Engineer.Dans un contexte : Datalake / Datalab :Être force de proposition pour permettre la meilleure performance des rapports pour les partenaires métiers, la partie Query, le modèle de données utilisé, la publication le rafraichissement, la disponibilité des données ;Participer à la production de rapports dans le cadre de projet ;Assurer la collecte des besoins auprès des partenaires métiers pour leur garantir un pilotage et une maîtrise de leur processus métier ;Participer à la récolte et à la conception de la répartition des données nécessaires aux rapports sur notre Datalake ;Rédiger une spécification technico fonctionnelle pour chaque reporting à produire et à monitorer ;Réaliser des tests de recette auprès de partenaires métiers, ainsi que de